In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import Normalizer
from scipy.io import arff
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten, LSTM
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(5,True,3)
cvscores = []

In [ ]:
ds= pd.read_csv('/content/drive/My Drive/code/dataset/df_spark.csv')
ds.shape
ds = ds.drop(columns="Unnamed: 0")
ds.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.0,21.0,30.0,2.0,3.0,30.0,2.0,3.0,59.0,7.0,2.0,0.0
1,7.0,32.0,70.0,2.0,4.0,68.0,2.0,4.0,134.0,7.0,2.0,0.0
2,7.0,15.0,1.0,2.0,2.0,0.0,2.0,2.0,0.0,7.0,2.0,0.0
3,7.0,33.0,77.0,2.0,7.0,73.0,2.0,7.0,144.0,7.0,2.0,0.0
4,7.0,55.0,78.0,3.0,7.0,74.0,3.0,7.0,146.0,8.0,2.0,0.0


In [ ]:
y = ds.iloc[:,0].values
X = ds.iloc[:,1:].values

from sklearn.preprocessing import label_binarize

y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7])
n_classes = y.shape[1]
n_classes

8

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

for train, test in kfold.split(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    x_train = np.array(X_train)
    x_test = np.array(X_test)
    y_train1 = np.array(y_train)
    y_test1 = np.array(y_test)

    model = Sequential()
    model.add(LSTM(4, input_dim=11))  # try using a GRU instead, for fun
    model.add(Dropout(0.6))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    #model.summary()

    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
    history = model.fit(X_train, y_train, epochs = 10, batch_size=64, verbose=1,shuffle=False)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/10
4475/4475 [==============================] - 8s 2ms/step - loss: 0.3850 - accuracy: 0.9639 - mae: 0.0520
Epoch 2/10
4475/4475 [==============================] - 8s 2ms/step - loss: 0.1210 - accuracy: 0.9784 - mae: 0.0120
Epoch 3/10
4475/4475 [==============================] - 8s 2ms/step - loss: 0.1068 - accuracy: 0.9795 - mae: 0.0096
Epoch 4/10
4475/4475 [==============================] - 8s 2ms/step - loss: 0.1011 - accuracy: 0.9800 - mae: 0.0091
Epoch 5/10
4475/4475 [==============================] - 8s 2ms/step - loss: 0.0979 - accuracy: 0.9803 - mae: 0.0088
Epoch 6/10
4475/4475 [==============================] - 8s 2ms/step - loss: 0.0953 - accuracy: 0.9808 - mae: 0.0086
Epoch 7/10
4475/4475 [==============================] - 8s 2ms/step - loss: 0.0933 - accuracy: 0.9810 - mae: 0.0085
Epoch 8/10
4475/4475 [==============================] - 8s 2ms/step - loss: 0.0932 - accuracy: 0.9810 - mae: 0.0085
Epoch 9/10
4475/4475 [==============================] - 8s 2ms/step - lo

In [ ]:
target_names = ['DoS', 'Probing',	'MaliciousControl', 'MaliciousOperation', 'Scan', 'Spying', 'WrongSetup', 'Normal']

In [ ]:
### train_accuracy
import sklearn.metrics as metrics
expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
#print("Cohens kappa:",metrics.cohen_kappa_score(expected, predicted))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

               DoS       0.91      0.66      0.77      4622
           Probing       0.00      0.00      0.00       257
  MaliciousControl       1.00      0.44      0.61       706
MaliciousOperation       0.00      0.00      0.00       645
              Scan       0.00      0.00      0.00      1249
            Spying       0.00      0.00      0.00       409
        WrongSetup       0.00      0.00      0.00       100
            Normal       0.99      1.00      0.99    278364

         micro avg       0.99      0.98      0.99    286352
         macro avg       0.36      0.26      0.30    286352
      weighted avg       0.98      0.98      0.98    286352
       samples avg       0.98      0.98      0.98    286352

Accuracy: 0.983621556685478


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.9778162268310965
Recall: 0.983621556685478
f1-score: 0.9800857538679428
ROC_AUC: 0.9851914792851493


In [ ]:
#### test accuracy
import sklearn.metrics as metrics
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)
# conmat = confusion_matrix(expected,predicted)
# print(conmat)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
#print("Cohens kappa:",metrics.cohen_kappa_score(expected, predicted))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

               DoS       0.91      0.63      0.75      1158
           Probing       0.00      0.00      0.00        85
  MaliciousControl       1.00      0.43      0.60       183
MaliciousOperation       0.00      0.00      0.00       160
              Scan       0.00      0.00      0.00       298
            Spying       0.00      0.00      0.00       123
        WrongSetup       0.00      0.00      0.00        22
            Normal       0.99      1.00      0.99     69560

         micro avg       0.99      0.98      0.98     71589
         macro avg       0.36      0.26      0.29     71589
      weighted avg       0.98      0.98      0.98     71589
       samples avg       0.98      0.98      0.98     71589

Accuracy: 0.9828744639539594
Precision: 0.9765337930201862
Recall: 0.9828744639539594
f1-score: 0.9789687156267497
ROC_AUC: 0.9857196179450827
